# A basic model to try and predict the Total Return of Eligible Stocks:
### Predicts the P/E multiple and hence the forecast price -> total return of a group of stocks

## Set-Up

### Import Packages

In [57]:
import os
import numpy as np
import pandas as pd
from scipy.optimize import root
import matplotlib.pyplot as plt

### Set Starting Parameters

In [6]:
R = 0.055
b = 1/(1+R)
maturity_g = 0.073
time_frame = 10
start_earnings = 10
__file__ = "NB04_Total_Return_Predictor.ipynb"
current_dir = os.path.dirname(os.path.abspath(__file__))


## Import the Bloomberg data from Excel

### Read In The Data

In [7]:
asset_df = pd.read_excel(os.path.join(current_dir, "../../data/Bloomberg_Rankings.xlsx"), sheet_name = "Mid Cap and Above")

### Inspect the Data

In [8]:
asset_df.head(8)

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.000000,300.470588,1.589450,13.526731,4.789175,1.168091e-02,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563
1,QNBFB TI Equity,QNB Finansbank AS,328.750000,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056
2,ADANIGR IN Equity,Adani Green Energy Ltd,2038.000000,63.698413,1.003621,6.944302,-74.834324,-3.671949e-02,12.776022,NaN,NaN,231.698702,0.004316,NaN,NaN,NaN,NaN
3,ADANI IN Equity,Adani Power Ltd,874.500000,46.398374,0.902365,54.003407,45.250976,5.174497e-02,94.187008,NaN,NaN,16.277971,0.061433,NaN,NaN,NaN,NaN
4,ADE IN Equity,Adani Enterprises Ltd,3645.250000,43.643866,0.883477,28.427895,-131.325263,-3.602641e-02,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944
5,SMCI US Equity,Super Micro Computer Inc,762.490723,30.004074,0.772433,19.230000,-33.824639,-4.436072e-02,72.776280,80.841010,80.841010,42.665360,0.023438,371.932307,0.487786,0.850796,1.045355
6,BYAN IJ Equity,Bayan Resources Tbk PT,18000.000000,22.218526,0.689037,0.034590,0.017968,9.982250e-07,-46.202778,63.130366,63.130366,32.120832,0.031132,0.399599,0.000022,0.682090,0.810109
7,6920 JP Equity,Lasertec Corp,40100.000000,21.450892,0.679599,742.560000,321.568675,8.019169e-03,105.626938,71.924868,71.924868,54.047060,0.018502,11153.856393,0.278151,0.751059,0.923087


### Drop the rows where "5Y Forecast EPS Growth" is Null
- We need this for our prediction of the P/E multiple, and then the EPS with which we multiply that multiple by to get the future price. 
- The current rudimentary form of predicting the multiple assumes that investors presume that EPS growth continues at the current rate for a certain set of years and then trails back down to a maturity growth rate. Investors purchase up until the point at which the multiple that they have priced in yields them the minimum expected rate of return, `R`, that they will take.
- I.e., the multiple today depends on the price and hence multiple tomorrow and so on so forth in a geometric series.
- Investors could calculate this `R` in any way, whether it be the CAPM or the Fama-French Three-Factor or another model to identify the best hurdle rate/ cost of equity capital they shoud take

In [9]:
asset_df = asset_df.dropna(subset=['5 YR Forecast EPS Growth'])
# Inspect the column (These figures will be in percentage, not proportion scale)
asset_df["5 YR Forecast EPS Growth"]


0       336.721739
1        92.741867
4        52.594889
5        80.841010
6        63.130366
           ...    
1033     12.294384
1039    -10.572361
1042      8.237525
1045     57.864844
1047     25.676972
Name: 5 YR Forecast EPS Growth, Length: 709, dtype: float64

In [10]:

def predict_multiple(growth, t): 
    growth = min([growth/100,0.30])
    maturity_g = 0.065 + 0.1*(growth - 0.065)
    k_vec_m = 1 + maturity_g
    maturity_multiple = (k_vec_m*b*(1-(k_vec_m*b)**18)/(1-b*k_vec_m))
    maturity_multiple   
    b_vec = [b] * t
    k_vec = [1 + growth] * t
    b_vec = np.cumprod(b_vec)[:t]
    k_vec = np.cumprod(k_vec)[:t]
   
    return sum(k_vec*b_vec) + maturity_multiple*b_vec[-1]*k_vec[-1]


predict_multiple(341.92,5)

np.float64(79.49405534513394)

In [11]:
asset_df["5Y Multiple"] = asset_df["5 YR Forecast EPS Growth"].apply(lambda x: predict_multiple(x, 5))


In [12]:
def predict_roi(growth, multiple, fut_multiple): 
    growth = min([growth/100,0.30])
    return (((1+growth)**6)*(1+(1/multiple)+(fut_multiple/multiple)))**1/6


## Predict the ROI Corresponding to this

In [13]:

asset_df["5Y Expected Annualised TR (Change in Multiple)"] = [predict_roi(asset_df["5 YR Forecast EPS Growth"][i], asset_df["Multiple"][i], asset_df["5Y Multiple"][i]) for i in asset_df.index]

In [14]:
asset_df.head()

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple),5Y Multiple,5Y Expected Annualised TR (Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.000000,300.470588,1.589450,13.526731,4.789175,1.168091e-02,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563,79.494055,2.301199
1,QNBFB TI Equity,QNB Finansbank AS,328.750000,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056,79.494055,2.944991
4,ADE IN Equity,Adani Enterprises Ltd,3645.250000,43.643866,0.883477,28.427895,-131.325263,-3.602641e-02,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944,79.494055,1.275612
5,SMCI US Equity,Super Micro Computer Inc,762.490723,30.004074,0.772433,19.230000,-33.824639,-4.436072e-02,72.776280,80.841010,80.841010,42.665360,0.023438,371.932307,0.487786,0.850796,1.045355,79.494055,2.322208
6,BYAN IJ Equity,Bayan Resources Tbk PT,18000.000000,22.218526,0.689037,0.034590,0.017968,9.982250e-07,-46.202778,63.130366,63.130366,32.120832,0.031132,0.399599,0.000022,0.682090,0.810109,79.494055,2.820447


In [15]:
asset_df.to_excel(os.path.join(current_dir,"../../data/Top_Stocks.xlsx"))

## Re-do with more complicated bloomberg data

### Import the data

In [48]:
df = pd.read_excel(os.path.join(current_dir, "../../data/bloomberg_data.xlsx"))

### Rename Some of the columns

In [49]:
df.rename(columns = {'Unnamed: 0': 'security_key'}, inplace = True)

### Take a deeper look at some of the columns, especially annualised performance over the last 5 years

In [50]:
pd.set_option('display.max_columns', None)
df.sort_values(by = "current_ann_trr_5yr", ascending = False)

,security_key,security_name,px_last,current_trr_ytd,cur_mkt_cap,current_trr_ytd_1,current_ann_trr_3yr,current_ann_trr_5yr,month_end_trr_10yr,minimum_total_return,hist_trr_prev_1yr,current_trr_mtd,eps_growth,geo_grow_diluted_eps_cont_ops,diluted_eps_cont_ops_5yr_avg_gr,5y_geo_growth_diluted_eps,earn_yld,pe_ratio,five_yr_avg_price_earnings,long_term_price_earnings_ratio,best_pe_ratio,pr_eps_growth_plus_yld
182,IHC UH Equity,International Holding Co PJSC,414.500,3.754693,9.092223e+11,3.754693,48.553680,209.09870,79.483020,NaN,-2.560967,1.593137,84.368610,196.464049,280.792101,215.717287,3.263385,30.643028,34.160428,NaN,NaN,NaN
240,NVDA US Equity,NVIDIA Corp,125.830,154.121900,3.095418e+12,154.121900,84.915230,100.09750,75.716730,NaN,239.019100,1.853651,584.659091,51.441971,168.466403,48.401189,1.364966,73.261887,61.097322,366.074157,43.136784,42.817484
309,TSLA US Equity,Tesla Inc,251.520,1.223442,8.021466e+11,1.223442,4.942127,74.82077,28.611223,NaN,101.721000,27.107340,17.661692,NaN,NaN,NaN,0.883704,113.160080,NaN,405.377228,94.449869,NaN
203,LLY US Equity,Eli Lilly & Co,914.570,57.423540,8.692123e+11,57.423540,58.498100,54.49960,33.149180,NaN,60.905030,1.015044,-16.017316,13.459286,22.494442,13.129725,1.133808,88.198379,38.449955,135.566601,62.526150,40.544444
199,KLAC US Equity,KLA Corp,855.210,47.740160,1.151454e+11,47.740160,42.903900,50.77105,33.369447,NaN,56.029640,3.723425,10.013593,25.827603,25.775781,36.480119,2.637979,37.907804,17.870855,62.973129,31.722616,9.645935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,P911 GR Equity,Dr Ing hc F Porsche AG,71.320,-7.935353,6.497252e+10,-7.935353,NaN,NaN,NaN,NaN,-14.915430,2.589187,4.044118,NaN,NaN,NaN,7.182267,13.923180,NaN,NaN,12.298672,6.568923
255,PLTR US Equity,Palantir Technologies Inc,27.230,58.590560,6.063839e+10,58.590560,5.661175,NaN,NaN,NaN,167.445400,7.500986,NaN,NaN,NaN,NaN,0.461469,216.699291,NaN,NaN,79.619883,NaN
258,PRX NA Equity,Prosus NV,32.835,21.678700,8.467185e+10,21.678700,-2.140393,NaN,NaN,NaN,-8.503771,-1.277809,62.145326,10.844301,75.982422,20.394921,7.447539,13.427254,18.448145,NaN,11.597772,46.123187
272,ROSN RM Equity,Rosneft Oil Co PJSC,556.150,NaN,5.894177e+12,NaN,NaN,NaN,14.568560,NaN,79.459110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.225822,NaN,NaN


### Define a function to apply some exclusion criteria
* Get rid of any stocks that were in the bottom 30% of performers in the 1yr, 3yr and 5yr annualised TR
* Get rid of any stocks with EPS growth in the last year, geometrically over 5 years or average over 5 years in the bottom 20 % of performers

In [88]:
def exclude(df, first_thresh = 30, second_thresh = 20):
    trr_cols = df.filter(regex='trr')
    for col in trr_cols:
        percentile = np.nanpercentile(df[col], 30)

        # Create a mask to filter rows where each 'trr' column is either NaN or >= the 50th percentile
        mask = (df[col].isna()) | (df[col] >= percentile)

        # Apply the mask to filter the DataFrame for each 'trr' column
        df = df[mask]
    eps_growth_cols = df.filter(regex='(gr.*eps|eps.*gr|gr.*earnings|earnings.*gr)')
    for col in eps_growth_cols:
        percentile = np.nanpercentile(df[col], 30)

        # Create a mask to filter rows where each 'trr' column is either NaN or >= the 50th percentile
        mask = (df[col].isna()) | (df[col] >= percentile)

        # Apply the mask to filter the DataFrame for each 'trr' column
        df = df[mask]
    

    return df

exclude(df)
    


,security_key,security_name,px_last,current_trr_ytd,cur_mkt_cap,current_trr_ytd_1,current_ann_trr_3yr,current_ann_trr_5yr,month_end_trr_10yr,minimum_total_return,hist_trr_prev_1yr,current_trr_mtd,eps_growth,geo_grow_diluted_eps_cont_ops,diluted_eps_cont_ops_5yr_avg_gr,5y_geo_growth_diluted_eps,earn_yld,pe_ratio,five_yr_avg_price_earnings,long_term_price_earnings_ratio,best_pe_ratio,pr_eps_growth_plus_yld
40,7974 JP Equity,Nintendo Co Ltd,8897.00,22.81723,1.155444e+13,22.81723,15.75242,20.77975,24.459750,NaN,38.01933,3.985507,13.456827,20.622616,33.361729,21.136470,4.736203,21.113959,16.599310,37.757079,30.504802,NaN
77,ANET US Equity,Arista Networks Inc,366.14,55.46687,1.147348e+11,55.46687,58.09722,40.39544,36.427448,NaN,94.07499,4.468159,53.061224,26.925057,36.606925,45.328895,1.957851,51.076409,37.389258,134.736412,45.807582,NaN
105,BREN IJ Equity,Barito Renewables Energy Tbk P,10300.00,37.83897,1.377998e+15,37.83897,NaN,NaN,NaN,NaN,NaN,2.233251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,GOOGL US Equity,Alphabet Inc,190.60,36.60091,2.363663e+12,36.60091,15.11437,27.82583,20.044368,NaN,58.32480,4.639040,27.233115,22.896718,33.493262,21.561293,3.468208,28.833333,26.285673,55.777962,23.656448,15.114932
226,MSFT US Equity,Microsoft Corp,467.56,24.78638,3.475049e+12,24.78638,20.00619,29.02111,28.622857,NaN,58.19189,4.611251,0.206186,21.778598,21.248099,35.474430,2.471377,40.463273,32.120102,69.569266,36.754972,15.431684
240,NVDA US Equity,NVIDIA Corp,125.83,154.12190,3.095418e+12,154.12190,84.91523,100.09750,75.716730,NaN,239.01910,1.853651,584.659091,51.441971,168.466403,48.401189,1.364966,73.261887,61.097322,366.074157,43.136784,42.817484
323,VGT US Equity,Vanguard Information Technolog,595.91,23.57851,7.719874e+10,23.57851,14.78231,23.75562,20.879360,-6.75760,52.65349,3.350725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,XLK US Equity,Technology Select Sector SPDR,232.88,21.40606,7.253185e+10,21.40606,16.81078,25.20144,20.971694,-6.27495,56.01636,2.939491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
